# Read Audio

In [214]:
import os
dir = os.getcwd()

import numpy as np

def get_audio_file(filename):

    # if filename.endswith('.mp3') or filename.endswith('.MP3'):
    #     import librosa
    #     sig, fs = librosa.load(dir + filename)
    # elif filename.endswith('.wav') or filename.endswith('.WAV'):
    #     from scipy.io import wavfile
    #     fs, sig = wavfile.read(dir + filename)
    # el
    if filename.endswith('.aif') or filename.endswith('.AIF'):
        import aifc
        audio = aifc.open(dir + filename)
    
        fs = audio.getframerate()
        nframes = audio.getnframes()
        rawsig = audio.readframes(nframes)
        sig = np.fromstring(rawsig, np.short).byteswap()

    return fs, sig

fsA, signalA = get_audio_file('/Audio/silentnight_piano.aif')
fsB, signalB = get_audio_file('/Audio/silentnight_guitar.aif')
fsC, signalC = get_audio_file('/Audio/littlestar_piano.aif')


C:\Users\elgor\AppData\Local\Temp/ipykernel_21144/1462662413.py:21: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  sig = np.fromstring(rawsig, np.short).byteswap()


# Get Frequency Components

In [216]:
from scipy import signal

fA, tA, ZxxA = signal.stft(signalA, fs=fsA)
magA = np.abs(ZxxA)

fB, tB, ZxxB = signal.stft(signalB, fs=fsB)
magB = np.abs(ZxxB)

fC, tC, ZxxC = signal.stft(signalC, fs=fsC)
magC = np.abs(ZxxC)
sphaseC = ZxxC/(magC + np.finfo(float).eps)

C:\Users\elgor\anaconda3\lib\site-packages\scipy\signal\spectral.py:1964: UserWarning: nperseg = 256 is greater than input length  = 2, using nperseg = 2
  warnings.warn('nperseg = {0:d} is greater than input length '


In [217]:
# import matplotlib.pyplot as plt

# plt.pcolormesh(tA, fA, np.abs(ZxxA), vmin=0, vmax=2 * np.sqrt(2), shading='gouraud')
# plt.title('STFT Magnitude')
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()

# Transform from Piano to Guitar

In [218]:
X =  np.matmul(magB, np.linalg.pinv(magA))

magC = magC.clip(min=0)

magD = np.matmul(X, magC)
_, rawsignalD = signal.istft(magD*sphaseC, fs=fsC)
signalD = np.rint(rawsignalD).astype(np.int16)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 129)

# Write Audio

In [ ]:
import scipy.io.wavfile as wav

wav.write('./Post_Audio/littlestar_guitar.wav', fsC, signalD.astype(np.int16))